# Limpieza y el preprocesamiento de los datos



In [2]:
import pandas as pd

# Defino los paths de los archivos Excel
pipomessage_path = "tecnica_diego/obscure_pipomessage.xlsx"
user_path = "tecnica_diego/obscure_user.xlsx"
usermessage_path = "tecnica_diego/usermessage.xlsx"
symptoms_path = "tecnica_diego/obscure_symptoms.xlsx"

# Cargar los datos en pandas DataFrames
pipomessage_df = pd.read_excel(pipomessage_path)
user_df = pd.read_excel(user_path)
usermessage_df = pd.read_excel(usermessage_path)
symptoms_df = pd.read_excel(symptoms_path)

Comenzaremos con la limpieza de los valores de intensidad de los síntomas. Asumimos que cualquier valor por encima de 10 es un error de entrada y debería haber sido 10. Entonces reemplazaremos estos valores con 10

In [3]:
# Replace symptom intensity values above 10 with 10
symptoms_df.loc[symptoms_df['intensity'] > 10, 'intensity'] = 10

# Verify the change
symptoms_df['intensity'].value_counts().sort_index()

intensity
0.0      19
1.0      80
2.0     355
3.0     527
4.0     461
5.0     652
6.0     483
7.0     523
8.0     617
9.0     294
10.0    290
Name: count, dtype: int64

# Valores faltantes
'cancer_type': Esta informacion solo se registra una sola vez. Entonces, la ausencia de un valor podría indicar que no se conoce el tipo de cáncer del usuario, por lo que podríamos reemplazar los valores faltantes con "Desconocido"

'body' de la tabla de mensajes de PIPO y de USER:  Vamos a eliminar las filas con mensajes faltantes, ya que es difícil imputar los mensajes

'intensity' de la tabla de registro de sintomas: Vamos a llenar los valores faltantes con la mediana, para preservar la distribución de los datos

In [5]:
from sklearn.impute import SimpleImputer

# Reemplazo los canceres faltantes con 'Desconocido'
user_df['cancer_type'].fillna('Desconocido', inplace=True)

# Eliminamos las filas con mensajes vacios
pipomessage_df.dropna(subset=['body'], inplace=True)
usermessage_df.dropna(subset=['body'], inplace=True)

# Intensidades faltantes con la mediana del dataset
imputer = SimpleImputer(strategy='median')
symptoms_df['intensity'] = imputer.fit_transform(symptoms_df[['intensity']])

# Pasamos a minusculas los mensajes para posterior tratamiento
pipomessage_df['body'] = pipomessage_df['body'].str.lower()
usermessage_df['body'] = usermessage_df['body'].str.lower()

# Verificamos
user_df.isnull().sum(), pipomessage_df.isnull().sum(), usermessage_df.isnull().sum(), symptoms_df.isnull().sum()


(phone_number     0
 creation_date    0
 cancer_type      0
 dtype: int64,
 id               0
 patient          0
 creation_date    0
 body             6
 dtype: int64,
 id                   0
 patient              1
 creation_date        0
 body              4534
 local_time       25080
 dtype: int64,
 id               0
 user             0
 creation_date    0
 description      0
 intensity        0
 dtype: int64)